## Temat 1. Własna sieć neuronowa
Celem projektu jest samodzielne zaimplementowanie prostej sieci neuronowej i przetestowanie jej na wybranym przez siebie przykładzie.

Podstawowym założeniem projektu jest własnoręczne zaimplementowanie automatycznego różniczkowania w celu wyznaczania pochodnych warstw potrzebnych do wstecznej   propagacji błędów. Dopuszczalne jest wykorzystanie różniczkowanie w przód (ang. Forward Accumulation), różniczkowanie w tył (ang. Reverse Accumulation) oraz różniczkowanie oparte o generację kodu (ang. source-to-source differentiation).

Do implementacji optymalizatorów dokonujących właściwego uczenia sieci można wykorzystać kod zamieszczony w książce "Algorithms for Optimization" [1].

klasyfikacja cyfr (Digits MNIST dataset)  
Bibliografia:  
[1] Mykel J. Kochenderfer, Tim A. Wheeler, 2019, Algorithms for Optimization, MIT Press.  
[2] Martin T. Hagan et. al, Neural Network Design, pp. 915-918, url: https://hagan.okstate.edu/NNDesign.pdf  
[3] 3blue1brown: czym są sieci neuronowe? https://www.youtube.com/watch?v=aircAruvnKk  
  


In [175]:
using Pkg
Pkg.activate("./project")

  Activating project at `e:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project`


In [176]:
Pkg.add("MLDatasets")
Pkg.add("IJulia")
Pkg.add("StableRNGs")
Pkg.add("DataFrames")
Pkg.add("PyPlot")
ENV["PYTHON"]=""

   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes 

to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


...

   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


""

# Load Dataset

In [177]:
using MLDatasets
using StableRNGs
train_x, train_y = MNIST.traindata(Float32);
test_x, test_y = MNIST.testdata(Float32);
Y_labels = [0,1,2,3,4,5,6,7,8,9]
train_size = size(train_x)
test_size = size(test_x)
train_x = reshape(train_x, (train_size[1],train_size[2],1,train_size[3]))
test_x = reshape(test_x, (test_size[1],test_size[2],1,test_size[3]))
println("Train X shape: ",size(train_x));
println("Test X shape: ",size((test_x)));
test_test_y = test_y
test_y= Y_labels  .== permutedims(test_y)
train_y= Y_labels .== permutedims(train_y)
println("Train Y shape: ",size(train_y));
println("Test Y shape: ",size((test_y)));

Train X shape: (28, 28, 1, 60000)
Test X shape: (28, 28, 1, 10000)
Train Y shape: (10, 60000)
Test Y shape: (10, 10000)


###  Automatic differentation- foward accumulation

In [178]:
# Dual number
struct Dual{T <:Number} <:Number
     v::T
    dv::T
end


### Overloading functions and operators

In [179]:
import Base: +, -, *, /
-(x::Dual)          = Dual(-x.v,       -x.dv)
+(x::Dual, y::Dual) = Dual( x.v + y.v,  x.dv + y.dv)
-(x::Dual, y::Dual) = Dual( x.v - y.v,  x.dv - y.dv)
*(x::Dual, y::Dual) = Dual( x.v * y.v,  x.dv * y.v + x.v * y.dv)
/(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)

import Base: abs, sin, cos, tan, exp, sqrt, isless, log, max, min
abs(x::Dual)  = Dual(abs(x.v),sign(x.v)*x.dv)
sin(x::Dual)  = Dual(sin(x.v), cos(x.v)*x.dv)
cos(x::Dual)  = Dual(cos(x.v),-sin(x.v)*x.dv)
tan(x::Dual)  = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
exp(x::Dual)  = Dual(exp(x.v), exp(x.v)*x.dv)
sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
isless(x::Dual, y::Dual) = x.v < y.v;
max(x::Dual, y::Dual) = max(x.v , y.v); # what aboud dv?
min(x::Dual, y::Dual) = min(x.v , y.v);
log(x::Dual) = Dual(log(x.v), (1/abs(x.v))*x.dv) 



log (generic function with 24 methods)

In [180]:
import Base: convert, promote_rule

convert(::Type{Dual{T}}, x::Dual) where T = Dual(convert(T, x.v), convert(T, x.dv))
@show Dual{Float64}[Dual(1,2), Dual(3,0)];
convert(::Type{Dual{T}}, x::Number) where T = Dual(convert(T, x), zero(T))
@show Dual{Float64}[1, 2, 3];
promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} = Dual{promote_type(T,R)}
@show Dual(1,2) * 3;

import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

Dual{Float64}[Dual(1, 2), Dual(3, 0)] = Dual{Float64}[(1.0) + [2.0ϵ], (3.0) + [0.0ϵ]]
Dual{Float64}[1, 2, 3] = Dual{Float64}[(1.0) + [0.0ϵ], (2.0) + [0.0ϵ], (3.0) + [0.0ϵ]]
Dual(1, 2) * 3 = (3) + [6ϵ]


In [181]:
D = derivative(f, x) = partials(f(Dual(x, one(x))))

derivative (generic function with 1 method)

In [182]:
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        column = partials.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [183]:
H = function hessian(f, args::Vector)
    ∇f(x::Vector) = J(f, x)
    J(∇f, args)
end

hessian (generic function with 1 method)

In [184]:
import LinearAlgebra: diagm
diagonal(m) = diagm(0 => vec(m))

diagonal (generic function with 1 method)

### Activation functions

In [185]:
softmax(x)  =  exp.(x) ./ sum(exp.(x));
dsoftmax(x) = (softmax(x) |> diagonal) .- softmax(x) * (softmax(x) |> transpose);

ReLU(x) = max(zero(x), x)
identityFunction(x) = x

σ(x) = one(x) / (one(x) + exp(-x))
tanh(x) = 2.0 / (one(x) + exp(-2.0x)) - one(x)

tanh (generic function with 1 method)

### Cost functions

In [186]:
mean_squared_loss(y::Vector, ŷ::Vector) = sum(0.5(y - ŷ).^2)

function binary_cross_entropy(y::Vector, ŷ::Vector) 
    epsilon = eps(1.0)
    ## Avoding 0 , 1 in log argument
    ŷ = [max(i, epsilon) for i in ŷ]
    ŷ = [min(i, 1-epsilon) for i in ŷ]
    return -sum(y .* log.(ŷ) + (1 .- y) .* log.(1 .- ŷ)) / length(y)
end

binary_cross_entropy (generic function with 1 method)

In [187]:
@show binary_cross_entropy([1,1,0.01],[1,1,0])
@show mean_squared_loss([1,1,1],[0,0,0])

binary_cross_entropy([1, 1, 0.01], [1, 1, 0]) = 0.1201455112970574
mean_squared_loss([1, 1, 1], [0, 0, 0]) = 1.5


1.5

### Net layers

In [188]:
fullyconnected(w::Vector, n::Number, m::Number, v::Vector, activation::Function) = activation.(reshape(w, n, m) * v)

fullyconnected (generic function with 1 method)

In [189]:
function net(x, wh, wo, y)
    x̂ = fullyconnected(wh, 10, 2, x, σ)
    ŷ = fullyconnected(wo, 1, 10, x̂, u->u)
    E = mean_squared_loss(y, ŷ)
end

net (generic function with 1 method)

### Network initialization

In [190]:


function net(x, wh, wo, y)
    x̂ = fullyconnected(wh, 16, 24, x, σ)
    ŷ = fullyconnected(wo, 9, 16, x̂, u -> u)
    E = mean_squared_loss(y, ŷ)
    return E
end


net (generic function with 1 method)

In [191]:
function foward(x, wh, wo, y)
    x̂ = fullyconnected(wh, 16, 24, x, σ)
    ŷ = fullyconnected(wo, 9, 16, x̂, u -> u)
    return ŷ
end

foward (generic function with 1 method)

In [192]:
struct Layer
    Wh::Matrix
    dWb::Matrix
    activation::Function
    Layer(in::Int64,out::Int64,activation::Function=identityFunction) = new(randn(in,out),randn(in,out),activation) #constructor
end

struct Network
    layers::Vector{Layer} 
    Network(layers::Vararg{Layer}) = new(vcat(layers...)) 
end

In [193]:
x  = randn(23,3)
length(x)

69

In [194]:
import Base: show, summary
show(io::IO, x::Network) = begin
    print("Neural Netowrk | ") 
    print("Number of layers : $(length(x.layers))") 
    println("\nLayers: ")
    for (i,layer) in enumerate(x.layers)
        print("\t$i. ")
        print(layer)
    end
end
show(io::IO, x::Layer) = begin
    print("Layer : ") 
    print("shape: $(size(x.Wh))") 
    println(" - activation function: $(x.activation)")
end

show (generic function with 352 methods)

In [195]:
(m::Layer)(x) = m.activation.(m.Wh*x) 
(m::Network)(x) = reduce((left,right)->right∘left,m.layers)(x)


In [201]:
x = [10, 4, 4, 8, 4, 1, 9, 0, 0, 1, 10, 3, 5, 3, 5, 4, 6, 5, 10, 6, 2, 10, 8, 1]   
y = [0,0,0,0,0,0,1,0,0]

twoLayerNeuralNet = Network(Layer(24,24,σ),Layer(9,24,σ)) #instantiate a two-layer network
for (i, layer) in enumerate(twoLayerNeuralNet.layers)
    x=layer(x)
    println(x)
end
L = mean_squared_loss(y, x)


[0.38702016743074713, 0.9999999999753668, 0.7586174903548171, 0.05061267343381272, 0.9999999995080562, 1.0, 0.9520045105509686, 0.9996748555675552, 0.9999543275093726, 0.26779068272377854, 1.010596345649264e-12, 6.078420135883192e-6, 0.17654442783098853, 4.020030163641174e-7, 7.843476088959065e-8, 1.1366028531695391e-7, 3.8058345309966866e-5, 4.528561594046657e-16, 0.9999999992811586, 0.9513084624673384, 7.087204648294812e-11, 3.1572658150188503e-6, 3.4512707155579907e-12, 1.2920850132141002e-33]
[0.6940371047105774, 0.18591858973217346, 0.3452496961204213, 0.9898106957684135, 0.4815791392207472, 0.35062784296583605, 0.3729617183755609, 0.8801727792813636, 0.47626472647087414]


1.6823716800690462

In [197]:
function train(epochs::Int, X_data, y_data)
    X_data = X_data
    y_data = y_data

    Wh = randn(16, 24)
    Wo = randn(9, 16)
    dWh = similar(Wh)
    dWo = similar(Wo)
    
    @assert size(X_data, 1) == size(y_data, 1)

    Loss_history = Float64[]

    for epoch = 1:epochs

        for (i, x) in enumerate(X_data)

            y = y_data[i]
            ŷ = foward(x, Wh[:], Wo[:], y[i])
            L = mean_squared_loss(y, ŷ)
        
            dnet_Wh(x, wh, wo, y) = J(w -> net(x, w, wo, y), wh)
            dWh[:] = dnet_Wh(x, Wh[:], Wo[:], y)
            dnet_Wo(x, wh, wo, y) = J(w -> net(x, wh, w, y), wo)
            dWo[:] = dnet_Wo(x, Wh[:], Wo[:], y)
        
            push!(Loss_history, L)

            Wh -= 0.1dWh
            Wo -= 0.1dWo
        end
    end
    return Loss_history
end

train (generic function with 1 method)

In [198]:
data = [[10, 4, 4, 8, 4, 1, 9, 0, 0, 1, 10, 3, 5, 3, 5, 4, 6, 5, 10, 6, 2, 10, 8, 1],
        [9, 2, 4, 7, 8, 5, 8, 1, 5, 4, 5, 8, 7, 5, 9, 9, 1, 9, 5, 10, 10, 0, 3, 3]]
y = [[0,0,0,0,0,0,1,0,0],[0,1,0,0,0,0,0,0,0]]
history=train(4,data,y)
print(history)

[16.540138471298917, 11.494876470909247, 3.3007083511711306, 1.4424595384434693, 0.7701906150582981, 0.5083567188577196, 0.5281771100843748, 0.5066547888446595]